<a href="https://colab.research.google.com/github/lilywangxuanlei/awesome-molecule-protein-pretrain-papers/blob/main/ECdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio torch-geometric
!pip install transformers
!pip install git+https://github.com/divelab/DIG.git@dig-stable

  Cloning https://github.com/divelab/DIG.git (to revision dig-stable) to /tmp/pip-req-build-xr_qhs9j
  Running command git clone --filter=blob:none --quiet https://github.com/divelab/DIG.git /tmp/pip-req-build-xr_qhs9j
  Resolved https://github.com/divelab/DIG.git to commit 21476b079c9226f38915dcd082b5c2ee0cddaac8
  Preparing metadata (setup.py) ... done


In [2]:
!pip install gdown


In [3]:
!gdown https://drive.google.com/uc?id=1DhZk5ykc2GtJNz25oYIgQl1P1Sr44hdL


Downloading...
From (original): https://drive.google.com/uc?id=1DhZk5ykc2GtJNz25oYIgQl1P1Sr44hdL
From (redirected): https://drive.google.com/uc?id=1DhZk5ykc2GtJNz25oYIgQl1P1Sr44hdL&confirm=t&uuid=221dd505-803e-4913-9d25-f9934d55037d
To: /content/ProtFunct.zip
100% 3.21G/3.21G [00:46<00:00, 68.4MB/s]


In [4]:
!unzip ProtFunct.zip -d ECdataset/

Streaming output truncated to the last 5000 lines.
  inflating: ECdataset/poolings/1oa9.A.hdf5  
  inflating: ECdataset/poolings/1oa7.A.hdf5  
  inflating: ECdataset/poolings/2eng.A.hdf5  
  inflating: ECdataset/poolings/3k65.A.hdf5  
  inflating: ECdataset/poolings/2e9l.A.hdf5  
  inflating: ECdataset/poolings/2e9m.A.hdf5  
  inflating: ECdataset/poolings/3v43.A.hdf5  
  inflating: ECdataset/poolings/3ru0.A.hdf5  
  inflating: ECdataset/poolings/3ru0.B.hdf5  
  inflating: ECdataset/poolings/4wzu.A.hdf5  
  inflating: ECdataset/poolings/3k54.A.hdf5  
  inflating: ECdataset/poolings/3v5j.A.hdf5  
  inflating: ECdataset/poolings/3v5l.C.hdf5  
  inflating: ECdataset/poolings/3v5l.D.hdf5  
  inflating: ECdataset/poolings/2gz2.A.hdf5  
  inflating: ECdataset/poolings/2gyy.B.hdf5  
  inflating: ECdataset/poolings/2gz3.C.hdf5  
  inflating: ECdataset/poolings/2gyy.A.hdf5  
  inflating: ECdataset/poolings/2gz1.B.hdf5  
  inflating: ECdataset/poolings/2yqb.A.hdf5  
  inflating: ECdataset/poolin

In [5]:
!mv ECdataset/training.txt ECdataset/split_train.txt
!mv ECdataset/validation.txt ECdataset/split_val.txt
!mv ECdataset/testing.txt ECdataset/split_test.txt

!ls ECdataset/

chain_functions.txt  processed	     split_train.txt
data		     raw	     split_val.txt
poolings	     split_test.txt  unique_functions.txt


In [6]:
!head ECdataset/split_train.txt


2pmo.X
2pmn.X
2kio.A
1w1n.A
2kit.A
5ygq.B
5ygq.A
3eb6.A
3t4n.A
2qlv.D


In [7]:
import h5py

path = 'ECdataset/data/102l.A.hdf5'  # use any file from your data folder

with h5py.File(path, 'r') as f:
    print("Keys:", list(f.keys()))


Keys: ['amino_chains', 'amino_neighs', 'amino_neighs_hb', 'amino_neighs_sindices', 'amino_neighs_sindices_hb', 'amino_pos', 'amino_types', 'atom_amino_id', 'atom_chain_ids', 'atom_chain_names', 'atom_names', 'atom_pos', 'atom_residue_id', 'atom_residue_names', 'atom_types', 'cov_bond_list', 'cov_bond_list_hb', 'cov_bond_list_sindices', 'cov_bond_list_sindices_hb', 'pos_center']


In [8]:
!ls ECdataset/data | head -n 10


102l.A.hdf5
103l.A.hdf5
104l.A.hdf5
104l.B.hdf5
107l.A.hdf5
108l.A.hdf5
109l.A.hdf5
110l.A.hdf5
111l.A.hdf5
112l.A.hdf5


In [9]:
!ls ECdataset/data/2pmo.X.pkl


ls: cannot access 'ECdataset/data/2pmo.X.pkl': No such file or directory


In [10]:
!head ECdataset/chain_functions.txt

4fae.B,247
3o9d.A,247
2rkg.B,247
3r0w.B,247
2qmp.B,247
2b60.B,247
3o9a.B,247
3m9f.A,247
2qmp.A,247
4m8y.B,247


In [15]:
import torch
from torch_geometric.data import InMemoryDataset, Data
import h5py
import os

class ProteinFunctionHDF5Dataset(InMemoryDataset):
    def __init__(self, root, split_file, function_file, transform=None, pre_transform=None):
        self.split_file = split_file
        self.function_file = function_file
        super(ProteinFunctionHDF5Dataset, self).__init__(root, transform, pre_transform)
        # 🚨 Important: weights_only=False to avoid UnpicklingError
        self.data, self.slices = torch.load(self.processed_paths[0], weights_only=False)

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['processed.pt']

    def download(self):
        pass

    def process(self):
        # Load function labels from chain_functions.txt
        func_dict = {}
        with open(self.function_file, 'r') as f:
            for line in f:
                name, label = line.strip().split(',')
                func_dict[name] = int(label)

        # Load split list
        with open(self.split_file, 'r') as f:
            proteins = [line.strip() for line in f]

        data_list = []

        for protein in proteins:
            filename = os.path.join(self.root, 'data', protein + '.hdf5')
            if not os.path.isfile(filename):
                print(f"❌ File missing: {filename}")
                continue

            try:
                with h5py.File(filename, 'r') as f:
                    # Load node features
                    amino_types = torch.tensor(f['amino_types'][:], dtype=torch.long)
                    if amino_types.size(0) == 0:
                        print(f"⚠️ Skipping {protein}: no amino acids.")
                        continue

                    # Load edge list
                    amino_neighs = torch.tensor(f['amino_neighs'][:], dtype=torch.long)
                    if amino_neighs.size(0) == 0:
                        print(f"⚠️ Skipping {protein}: no neighbors.")
                        continue

                    # Filter invalid neighbor indices
                    valid_mask = (amino_neighs >= 0) & (amino_neighs < amino_types.size(0))
                    valid_rows = valid_mask.all(dim=1)
                    amino_neighs = amino_neighs[valid_rows]

                    if amino_neighs.size(0) == 0:
                        print(f"⚠️ Skipping {protein}: no valid neighbors after filtering.")
                        continue

                    edge_index = amino_neighs.t()  # transpose to [2, num_edges]

                    # One-hot encode amino acid types
                    x = torch.nn.functional.one_hot(amino_types, num_classes=21).float()

                    # Lookup label
                    if protein not in func_dict:
                        print(f"⚠️ Skipping {protein}: no label found.")
                        continue

                    label = func_dict[protein]
                    if label < 0:
                        print(f"⚠️ Skipping {protein}: invalid (negative) label {label}.")
                        continue

                    y = torch.tensor([label], dtype=torch.long)

                    # Build graph
                    data = Data(x=x, edge_index=edge_index, y=y)
                    data_list.append(data)

            except Exception as e:
                print(f"❌ Error loading {protein}: {e}")
                continue

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


In [16]:
path = 'ECdataset/data/3jxv.A.hdf5'  # use any file from your data folder

with h5py.File(path, 'r') as f:
    print("Keys:", list(f.keys()))


Keys: ['amino_chains', 'amino_neighs', 'amino_neighs_hb', 'amino_neighs_sindices', 'amino_neighs_sindices_hb', 'amino_pos', 'amino_types', 'atom_amino_id', 'atom_chain_ids', 'atom_chain_names', 'atom_names', 'atom_pos', 'atom_residue_id', 'atom_residue_names', 'atom_types', 'cov_bond_list', 'cov_bond_list_hb', 'cov_bond_list_sindices', 'cov_bond_list_sindices_hb', 'pos_center']


In [17]:
dataset = ProteinFunctionHDF5Dataset(
    root='./ECdataset',
    split_file='./ECdataset/split_train.txt',
    function_file='./ECdataset/chain_functions.txt'
)

print(f"Successfully loaded {len(dataset)} protein graphs!")
print(dataset[0])


Successfully loaded 29171 protein graphs!
Data(x=[340, 21], edge_index=[2, 676], y=[1])


In [18]:

# Print how many samples we have
print(f"Dataset size: {len(dataset)}")

# Look at one sample
sample = dataset[0]
print(sample)

Dataset size: 29171
Data(x=[340, 21], edge_index=[2, 676], y=[1])


In [20]:
embedding_dim = 768
fc = torch.nn.Linear(21, embedding_dim)


In [22]:
# Get one graph from your dataset
graph = dataset[0]

# Map node features (21-dim) to ProSST embedding dim (768)
fc = torch.nn.Linear(21, 768)  # define this only once

# Embed the node features
inputs_embeds = fc(graph.x).unsqueeze(0)  # (1, num_nodes, 768)


In [27]:
graph = dataset[0]  # one graph

input_ids = graph.x.argmax(dim=-1).unsqueeze(0)  # (1, num_nodes)

from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained(
    "AI4Protein/ProSST-Foldseek",
    trust_remote_code=True
)

outputs = model(input_ids=input_ids)
print(outputs.last_hidden_state.shape)  # (1, num_nodes, 768)


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not NoneType